# Combinar conjuntos de datos: Merge y join

Una característica esencial que ofrece Pandas son sus operaciones join y merge de alto rendimiento en memoria.
Si alguna vez has trabajado con bases de datos, deberías estar familiarizado con este tipo de interacción de datos.
La interfaz principal para esto es la función ``pd.merge``, y veremos algunos ejemplos de cómo puede funcionar en la práctica.

Por conveniencia, empezaremos redefiniendo la funcionalidad ``display()`` de la sección anterior:

In [ ]:
import pandas as pd
import numpy as np

class display(object):
    """Mostrar la representación HTML de varios objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Álgebra Relacional

El comportamiento implementado en ``pd.merge()`` es un subconjunto de lo que se conoce como *álgebra relacional*, que es un conjunto formal de reglas para manipular datos relacionales, y forma la base conceptual de las operaciones disponibles en la mayoría de las bases de datos.
El punto fuerte del enfoque del álgebra relacional es que propone varias operaciones primitivas, que se convierten en los bloques de construcción de operaciones más complicadas sobre cualquier conjunto de datos.
Con este léxico de operaciones fundamentales implementado de forma eficiente en una base de datos u otro programa, se puede realizar una amplia gama de operaciones compuestas bastante complicadas.

Pandas implementa varios de estos bloques de construcción fundamentales en la función ``pd.merge()`` y el método relacionado ``join()`` de ``Series`` y ``Dataframe``.
Como veremos, estos métodos permiten enlazar datos de diferentes fuentes.

## Categorías de uniones

La función ``pd.merge()`` implementa varios tipos de uniones: las uniones *uno-a-uno*, *many-a-uno* y *many-a-muchos*.
A los tres tipos de uniones se accede mediante una llamada idéntica a la interfaz ``pd.merge()``; el tipo de unión realizada depende de la forma de los datos de entrada.
Aquí mostraremos ejemplos sencillos de los tres tipos de uniones, y discutiremos las opciones detalladas más adelante.

### Uniones uno a uno

Quizá el tipo más sencillo de expresión de fusión sea la unión uno a uno, que en muchos aspectos es muy similar a la concatenación por columnas vista en [Combining Datasets: Concat & Append](6_Concat-And-Append.ipynb).
Como ejemplo concreto, consideremos los dos ``DataFrames`` siguientes, que contienen información sobre varios empleados de una empresa:

In [ ]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

Para combinar esta información en un único ``DataFrame``, podemos utilizar la función ``pd.merge()``:

In [ ]:
df3 = pd.merge(df1, df2)
df3

La función ``pd.merge()`` reconoce que cada ``DataFrame`` tiene una columna "empleado", y une automáticamente utilizando esta columna como clave.

El resultado de la fusión es un nuevo ``DataFrame`` que combina la información de las dos entradas.
Observa que el orden de las entradas en cada columna no se mantiene necesariamente: en este caso, el orden de la columna "empleado" difiere entre ``df1`` y ``df2``, y la función ``pd.merge()`` lo tiene en cuenta correctamente.

Además, hay que tener en cuenta que la fusión en general descarta el índice, excepto en el caso especial de las fusiones por índice (véanse las palabras clave ``left_index`` y ``right_index``, de las que hablaremos más adelante).

### Many-to-one joins

Las uniones múltiples son uniones en las que una de las dos columnas clave contiene entradas duplicadas.
En este caso, el ``DataFrame`` resultante conservará las entradas duplicadas.
Considere el siguiente ejemplo de unión muchos-a-uno:

In [ ]:
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR',],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3', 'df4', 'pd.merge(df3, df4)')

El ``DataFrame`` resultante tiene una columna adicional con la información del "supervisor", donde la información se repite en uno o más lugares según lo requieran las entradas.

### Many-to-many joins

Las uniones muchos-a-muchos son un poco confusas conceptualmente, pero sin embargo están bien definidas.
Si la columna clave tanto de la matriz izquierda como de la derecha contiene duplicados, el resultado es una fusión de muchos a muchos.

Quizá esto quede más claro con un ejemplo concreto.

Considere lo siguiente, donde tenemos un ``DataFrame`` que muestra una o más habilidades asociadas con un grupo en particular.

Realizando una unión de muchos a muchos, podemos recuperar las habilidades asociadas a cualquier persona individual:

In [ ]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5', "pd.merge(df1, df5)")

Estos tres tipos de uniones se pueden utilizar con otras herramientas de Pandas para implementar una amplia gama de funcionalidades.

Pero en la práctica, los conjuntos de datos raramente están tan limpios como con el que estamos trabajando aquí.

En la siguiente sección consideraremos algunas de las opciones proporcionadas por ``pd.merge()`` que permiten ajustar el funcionamiento de las operaciones de unión.

## Especificación de la clave de fusión

Ya hemos visto el comportamiento por defecto de ``pd.merge()``: busca uno o más nombres de columna que coincidan entre las dos entradas y los utiliza como clave.

Sin embargo, a menudo los nombres de las columnas no coinciden tan bien, y ``pd.merge()`` proporciona una variedad de opciones para manejar esto.

### La palabra clave ``on``

Lo más sencillo es especificar explícitamente el nombre de la columna clave mediante la palabra clave ``on``, que toma un nombre de columna o una lista de nombres de columna:

In [ ]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

Esta opción sólo funciona si tanto el ``DataFrame`` izquierdo como el derecho tienen el nombre de columna especificado.

### Las palabras clave ``left_on`` y ``right_on``

En ocasiones, es posible que desee combinar dos conjuntos de datos con nombres de columna diferentes; por ejemplo, podemos tener un conjunto de datos en el que el nombre del empleado esté etiquetado como "nombre" en lugar de "empleado".
En este caso, podemos utilizar las palabras clave ``left_on`` y ``right_on`` para especificar los dos nombres de columna:

In [ ]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})
display('df1', 'df3', 'pd.merge(df1, df3, left_on="employee", right_on="name")')

El resultado tiene una columna redundante que podemos eliminar si lo deseamos, por ejemplo, utilizando el método ``drop()`` de ``DataFrame``s:

In [ ]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

### Las palabras clave ``left_index`` y ``right_index``

A veces, en lugar de fusionar en una columna, se desea hacerlo en un índice.
Por ejemplo, sus datos podrían tener este aspecto:

In [ ]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

Puedes utilizar el índice como clave para la fusión especificando los indicadores ``left_index`` y/o ``right_index`` en ``pd.merge()``:

In [ ]:
display('df1a', 'df2a',
        "pd.merge(df1a, df2a, left_index=True, right_index=True)")

Por comodidad, los ``DataFrame`` implementan el método ``join()``, que realiza una fusión que por defecto une los índices:

In [ ]:
display('df1a', 'df2a', 'df1a.join(df2a)')

Si quieres mezclar índices y columnas, puedes combinar ``left_index`` con ``right_on`` o ``left_on`` con ``right_index`` para obtener el comportamiento deseado:

In [ ]:
display('df1a', 'df3', "pd.merge(df1a, df3, left_index=True, right_on='name')")

Todas estas opciones también funcionan con múltiples índices y/o múltiples columnas; la interfaz para este comportamiento es muy intuitiva.
Para más información sobre esto, consulte la sección ["Merge, Join, and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) de la documentación de Pandas.

## Especificación de la aritmética de conjuntos para uniones

En todos los ejemplos anteriores hemos pasado por alto una consideración importante a la hora de realizar una unión: el tipo de aritmética de conjuntos utilizado en la unión.
Esto ocurre cuando un valor aparece en una columna clave pero no en la otra. Veamos este ejemplo:

In [ ]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', 'pd.merge(df6, df7)')

Aquí hemos fusionado dos conjuntos de datos que sólo tienen una entrada de "nombre" en común: María.
Por defecto, el resultado contiene la *intersección* de los dos conjuntos de entradas; esto es lo que se conoce como *inner join*.

Podemos especificarlo explícitamente mediante la palabra clave ``how``, que por defecto es ``"inner"``:

In [ ]:
pd.merge(df6, df7, how='inner')

Otras opciones para la palabra clave ``how`` son ``'outer``, ``'left`` y ``'right``.
Un *outer join* devuelve un join sobre la unión de las columnas de entrada, y rellena todos los valores perdidos con NAs:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='outer')")

Las opciones *left join* y *right join* devuelven uniones sobre las entradas de la izquierda y la derecha, respectivamente.
Por ejemplo:

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

In [ ]:
display('df6', 'df7', "pd.merge(df6, df7, how='right')")

Las filas de salida corresponden ahora a las entradas de la entrada izquierda. Utilizando
``how='right'`` funciona de forma similar.

Todas estas opciones pueden aplicarse directamente a cualquiera de los tipos de unión anteriores.

## Nombres de columnas solapados: La palabra clave ``suffixes``

Por último, puede darse el caso de que sus dos ``DataFrame`` de entrada tengan nombres de columna contradictorios.
Considere este ejemplo:

In [ ]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9', 'pd.merge(df8, df9, on="name")')

Dado que la salida tendría dos nombres de columna en conflicto, la función de fusión añade automáticamente un sufijo ``_x`` o ``_y`` para hacer que las columnas de salida sean únicas.

Si estos valores por defecto no son apropiados, es posible especificar un sufijo personalizado utilizando la palabra clave ``suffixes``:

In [ ]:
display('df8', 'df9', 'pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])')

Estos sufijos funcionan en cualquiera de los posibles patrones de unión, y funcionan también si hay varias columnas superpuestas.

Para más información sobre estos patrones, vea [Aggregation and Grouping](8_Aggregation-and-Grouping.ipynb) donde profundizamos un poco más en el álgebra relacional.
Consulte también la documentación de [Pandas "Merge, Join and Concatenate"](http://pandas.pydata.org/pandas-docs/stable/merging.html) para más información sobre estos temas.

## Ejemplo: Datos de los Estados de EE.UU.

Las operaciones merge y join aparecen con más frecuencia cuando se combinan datos de distintas fuentes.
Aquí consideraremos un ejemplo de algunos datos sobre los estados de EE.UU. y su población.
Los archivos de datos se encuentran en http://github.com/jakevdp/data-USstates/:

In [ ]:
# Los siguientes son comandos shell para descargar los datos
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
# !curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

Echemos un vistazo a los tres conjuntos de datos, utilizando la función Pandas ``read_csv()``:

In [ ]:
pop = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

Con esta información, supongamos que queremos calcular un resultado relativamente sencillo: clasificar los estados y territorios de EE.UU. según su densidad de población en 2010.
Es evidente que disponemos de los datos necesarios para obtener este resultado, pero tendremos que combinar los conjuntos de datos para obtenerlo.

Empezaremos con una combinación múltiple que nos dará el nombre completo del estado dentro del ``DataFrame`` de población.

Queremos hacer la combinación basándonos en la columna ``state/region`` de ``pop`` y en la columna ``abbreviation`` de ``abbrevs``.

Utilizaremos ``how='outer'`` para asegurarnos de que no se pierde ningún dato por no coincidir las etiquetas.

In [ ]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # drop duplicate info
merged.head()

Comprobemos si se ha producido alguna discordancia, lo que podemos hacer buscando filas con nulos:

In [ ]:
merged.isnull().any()

Algunos de los datos de ``población`` son nulos; ¡averigüemos cuáles son!

In [ ]:
merged[merged['population'].isnull()].head()

Parece que todos los valores nulos de población son de Puerto Rico anteriores al año 2000; esto se debe probablemente a que estos datos no están disponibles en la fuente original.

Y lo que es más importante, vemos también que algunas de las nuevas entradas ``state`` también son nulas, lo que significa que no había ninguna entrada correspondiente en la clave ``abbrevs``.
Averigüemos qué regiones carecen de esta coincidencia:

In [ ]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

Podemos deducir rápidamente el problema: nuestros datos de población incluyen entradas para Puerto Rico (PR) y los Estados Unidos en su conjunto (USA), mientras que estas entradas no aparecen en la clave de abreviatura del estado.
Podemos solucionarlo rápidamente introduciendo las entradas adecuadas:

In [ ]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

Se acabaron los nulos en la columna ``state``: ¡listo!

Ahora podemos fusionar el resultado con los datos del área mediante un procedimiento similar.
Al examinar nuestros resultados, querremos unirlos en la columna ``state`` de ambos:

In [ ]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

De nuevo, vamos a comprobar si hay nulos para ver si hay algún desajuste:

In [ ]:
final.isnull().any()

Hay nulos en la columna ``área``; podemos echar un vistazo para ver qué regiones se ignoraron aquí:

In [ ]:
final['state'][final['area (sq. mi)'].isnull()].unique()

Vemos que nuestro ``DataFrame`` de ``areas`` no contiene el área de Estados Unidos en su conjunto.
Podríamos insertar el valor apropiado (usando la suma de las áreas de todos los estados, por ejemplo), pero en este caso simplemente eliminaremos los valores nulos porque la densidad de población de todo Estados Unidos no es relevante para nuestra discusión actual:

In [ ]:
final.dropna(inplace=True)
final.head()

Ahora tenemos todos los datos que necesitamos. Para responder a la pregunta que nos interesa, vamos a seleccionar primero la parte de los datos correspondiente al año 2000 y la población total.
Usaremos la función ``query()`` para hacerlo rápidamente (esto requiere tener instalado el paquete ``numexpr``; ver [Pandas de alto rendimiento: ``eval()`` y ``query()``](12_Performance-Eval-and-Query.ipynb)):

In [ ]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

Ahora vamos a calcular la densidad de población y mostrarla por orden.
Empezaremos por volver a indexar nuestros datos en el estado y, a continuación, calcularemos el resultado:

In [ ]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [ ]:
density.sort_values(ascending=False, inplace=True)
density.head()

El resultado es una clasificación de los estados de EE.UU. más Washington, DC, y Puerto Rico por orden de densidad de población en 2010, en residentes por milla cuadrada.
Podemos ver que, con diferencia, la región más densa en este conjunto de datos es Washington, DC (es decir, el Distrito de Columbia); entre los estados, el más denso es Nueva Jersey.

También podemos comprobar el final de la lista:

In [ ]:
density.tail()

Vemos que el estado menos denso, con diferencia, es Alaska, con una media de poco más de un residente por milla cuadrada.

Este tipo de fusión de datos desordenados es una tarea habitual cuando se intenta responder a preguntas utilizando fuentes de datos del mundo real.
Espero que este ejemplo te haya dado una idea de las formas en que puedes combinar las herramientas que hemos visto para obtener información de tus datos.